In [16]:
!pip install torch==1.3.1

In [0]:
from zipfile import ZipFile
import pathlib
from urllib.request import urlretrieve
import os
import time
from os import remove
import os.path
import copy
import numpy as np
import pandas as pd
import shutil as sh
import torch
import glob
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
FID_DIRECTORY = pathlib.Path("FID-300")
FID_LABELS = FID_DIRECTORY / "label_table.csv"
FID_SOURCE_URL = "https://fid.dmi.unibas.ch/FID-300.zip"
TRAIN_DIR = "FID-300/tracks_cropped/"
NUM_EPOCHS = 1000
BATCH_SIZE = 64
NUM_CLASSES = 130
INPUT_SIZE = 224
MODEL_NAME = "vgg"
FEATURE_EXTRACT=False
USE_PRETRAINED=False

In [0]:
def process_images(valid_size=0.1):
    transformations = transforms.Compose([
        transforms.RandomResizedCrop(INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                             0.229, 0.224, 0.225])
    ])

    train_data = datasets.ImageFolder(TRAIN_DIR, transform=transformations)
    test_data = datasets.ImageFolder(TRAIN_DIR, transform=transformations)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)

    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                                              sampler=train_sampler, batch_size=64)
    testloader = torch.utils.data.DataLoader(test_data,
                                             sampler=test_sampler, batch_size=64)
    dataloaders_dict = {'train':trainloader, 'val':testloader}
    return dataloaders_dict

In [0]:
def organize_files(label_df):
    """ Moves all pngs in tracked_cropped into subfolders by label (for PyTorch Image Folder) """
    train_dir = pathlib.Path(TRAIN_DIR)

    files = glob.glob(str(train_dir / "*.jpg"))
    for i in range(len(files)):
        f = pathlib.Path(files[i])
        fname = f.name
        id = int(f.stem)
        label = label_df["label"].iloc[id-1]
        new_dir = train_dir / str(label)
        new_dir.mkdir(exist_ok=True)
        new_file = new_dir / fname

        sh.move(f, new_file)

In [0]:
def load_labels():
    labels = pd.read_csv(FID_LABELS, delimiter=",", header=None,
                         dtype=np.dtype(int), names=['id', 'label'])
    # print(len(pd.unique(labels['label'])))
    return labels


def fetch_FID_300_data():
    """Downloads and extracts FID-300 data to a local folder"""
    if FID_DIRECTORY.exists():
        print("FID-300 Database already exists")
        return
    print("Downloading FID_300")
    local_file, _ = urlretrieve(FID_SOURCE_URL)
    with ZipFile(local_file) as archive:
        print("Extracting FID-300")
        archive.extractall()
    remove(local_file)

In [0]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [23]:
def initialize_model(MODEL_NAME, NUM_CLASSES, FEATURE_EXTRACT=False, USE_PRETRAINED=False):
    model_ft = None
    input_size = 0

    if MODEL_NAME == "resnet":
        model_ft = models.resnet18(pretrained=USE_PRETRAINED)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, NUM_CLASSES)
        input_size = 224

    elif MODEL_NAME == "alexnet":
        model_ft = models.alexnet(pretrained=USE_PRETRAINED)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,NUM_CLASSES)
        input_size = 224

    elif MODEL_NAME == "vgg":
        model_ft = models.vgg11_bn(pretrained=USE_PRETRAINED)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,NUM_CLASSES)
        input_size = 224

    elif MODEL_NAME == "squeezenet":
        model_ft = models.squeezenet1_0(pretrained=USE_PRETRAINED)
        model_ft.classifier[1] = nn.Conv2d(512, NUM_CLASSES, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = NUM_CLASSES
        input_size = 224

    elif MODEL_NAME == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=USE_PRETRAINED)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, NUM_CLASSES)
        input_size = 224

    elif MODEL_NAME == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=USE_PRETRAINED)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, NUM_CLASSES)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,NUM_CLASSES)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(MODEL_NAME, NUM_CLASSES, 
                                        FEATURE_EXTRACT, USE_PRETRAINED)

# Print the model we just instantiated
print(model_ft)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if FEATURE_EXTRACT:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 features.0.weight
	 features.0.bias
	 features.1.weight
	 features.1.bias
	 features.4.weight
	 features.4.bias
	 features.5.weight
	 features.5.bias
	 features.8.weight
	 features.8.bias
	 features.9.weight
	 features.9.bias
	 features.11.weight
	 features.11.bias
	 features.12.weight
	 features.12.bias
	 features.15.weight
	 features.15.bias
	 features.16.weight
	 features.16.bias
	 features.18.weight
	 features.18.bias
	 features.19.weight
	 features.19.bias
	 features.22.weight
	 features.22.bias
	 features.23.weight
	 features.23.bias
	 features.25.weight
	 features.25.bias
	 features.26.weight
	 features.26.bias
	 classifier.0.weight
	 classifier.0.bias
	 classifier.3.weight
	 classifier.3.bias
	 classifier.6.weight
	 classifier.6.bias


In [25]:
fetch_FID_300_data()
labels = load_labels()
organize_files(labels)
dataloaders_dict = process_images()

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, 
                             num_epochs=NUM_EPOCHS, is_inception=(MODEL_NAME=="inception"))

FID-300 Database already exists
Epoch 0/999
----------
train Loss: 4.5665 Acc: 0.0067
val Loss: 0.4879 Acc: 0.0000

Epoch 1/999
----------
train Loss: 4.5029 Acc: 0.0100
val Loss: 0.4867 Acc: 0.0000

Epoch 2/999
----------
train Loss: 4.3524 Acc: 0.0233
val Loss: 0.4870 Acc: 0.0000

Epoch 3/999
----------
train Loss: 4.3633 Acc: 0.0200
val Loss: 0.4889 Acc: 0.0033

Epoch 4/999
----------
train Loss: 4.3349 Acc: 0.0267
val Loss: 0.4923 Acc: 0.0000

Epoch 5/999
----------
train Loss: 4.2662 Acc: 0.0267
val Loss: 0.4951 Acc: 0.0000

Epoch 6/999
----------
train Loss: 4.2086 Acc: 0.0233
val Loss: 0.4993 Acc: 0.0000

Epoch 7/999
----------
train Loss: 4.2517 Acc: 0.0200
val Loss: 0.5022 Acc: 0.0000

Epoch 8/999
----------
train Loss: 4.2198 Acc: 0.0200
val Loss: 0.5098 Acc: 0.0000

Epoch 9/999
----------
train Loss: 4.1492 Acc: 0.0133
val Loss: 0.5128 Acc: 0.0000

Epoch 10/999
----------
train Loss: 4.1524 Acc: 0.0200
val Loss: 0.5256 Acc: 0.0000

Epoch 11/999
----------
train Loss: 4.1679 